In [67]:
# ruff: noqa
import pickle

from datasets import ClassLabel, Dataset, concatenate_datasets, load_dataset

from divide_and_conquer_sentiment.aggregation import MLP, MLPAggregator
from divide_and_conquer_sentiment.dataloaders import load_kaggle_dataset
from divide_and_conquer_sentiment.evaluation import compare_multiple, simple_classification_report
from divide_and_conquer_sentiment.model import DACSModel
from divide_and_conquer_sentiment.subprediction import ABSASubpredictor

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TRAIN_BATCH_SIZE = 256

# Read & prepare dataset


In [4]:
twitter_airlines_dataset = load_kaggle_dataset(
    "crowdflower/twitter-airline-sentiment",
    {"text": "text", "airline_sentiment": "label"},
    val_test_perc=(0.1, 0.2),
    seed=42,
)

Dataset URL: https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment


/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/kaggle/api_client.py:165: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  response_data.getheaders())


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/14640 [00:00<?, ? examples/s]

Map:   0%|          | 0/14640 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/14640 [00:00<?, ? examples/s]

In [5]:
amazon_headphones_dataset = load_kaggle_dataset(
    "mdwaquarazam/headphone-dataset-review-analysis",
    {"COMMENTS": "text", "RATINGS": "label"},
    val_test_perc=(0.1, 0.2),
    seed=42,
)

Dataset URL: https://www.kaggle.com/datasets/mdwaquarazam/headphone-dataset-review-analysis


/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/kaggle/api_client.py:165: DeprecationWarning: HTTPResponse.getheaders() is deprecated and will be removed in urllib3 v2.1.0. Instead access HTTPResponse.headers directly.
  response_data.getheaders())


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1604 [00:00<?, ? examples/s]

Map:   0%|          | 0/1604 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1604 [00:00<?, ? examples/s]

In [6]:
SST_LABEL_MAP = {0: 0, 1: 0, 2: 1, 3: 2, 4: 2}


def map_sst_label(x):
    x["label"] = SST_LABEL_MAP[x["label"]]
    return x


sst_dataset = (
    load_dataset("SetFit/sst5")
    .remove_columns(["label_text"])
    .map(map_sst_label)
    .cast_column("label", ClassLabel(names=["negative", "neutral", "positive"]))
)

Repo card metadata block was not found. Setting CardData to empty.


In [7]:
train_dataset = concatenate_datasets(
    [twitter_airlines_dataset["train"], amazon_headphones_dataset["train"], sst_dataset["train"]]
)
val_dataset = concatenate_datasets(
    [twitter_airlines_dataset["val"], amazon_headphones_dataset["val"], sst_dataset["validation"]]
)

# ABSA

## Train MLP

In [8]:
!spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 30.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [8]:
subpredictor = ABSASubpredictor.from_pretrained(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
    spacy_model="en_core_web_lg",
)
mlp = MLP(input_size=4, output_size=3, hidden_layer_sizes=(128, 64))
aggregator = MLPAggregator(mlp)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect
/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: mps
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, u

In [ ]:
train_subpreds = subpredictor.predict(train_dataset["text"])
val_subpreds = subpredictor.predict(val_dataset["text"])

In [18]:
train_dataset = concatenate_datasets([train_dataset, Dataset.from_dict({"subpreds": train_subpreds})], axis=1)
val_dataset = concatenate_datasets([val_dataset, Dataset.from_dict({"subpreds": val_subpreds})], axis=1)

ValueError: The table can't have duplicated columns but columns ['subpreds'] are duplicated.

In [20]:
with open("train_dataset_subpreds.pkl", "wb") as handle:
    pickle.dump(train_dataset, handle)

with open("val_dataset_subpreds.pkl", "wb") as handle:
    pickle.dump(val_dataset, handle)

In [21]:
with open("train_dataset_subpreds.pkl", "rb") as handle:
    train_dataset = pickle.load(handle).with_format("torch")

with open("val_dataset_subpreds.pkl", "rb") as handle:
    val_dataset = pickle.load(handle).with_format("torch")

In [22]:
aggregator.train(train_dataset, val_dataset)


  | Name   | Type       | Params | Mode
---------------------------------------------
0 | layers | ModuleList | 11.3 K | eval
---------------------------------------------
11.3 K    Trainable params
0         Non-trainable params
11.3 K    Total params
0.045     Total estimated model params size (MB)
0         Modules in train mode
4         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/jankoscialkowski/Documents/src/divide-and-conquer-sentiment/venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## Evaluate

In [23]:
dacs_absa = DACSModel(subpredictor, aggregator)

In [24]:
twitter_test_dacs_preds = dacs_absa.predict(twitter_airlines_dataset["test"]["text"])
amazon_test_dacs_preds = dacs_absa.predict(amazon_headphones_dataset["test"]["text"])
sst_test_dacs_preds = dacs_absa.predict(sst_dataset["test"]["text"])

Batches:   0%|          | 0/391 [00:00<?, ?it/s]

Batches:   0%|          | 0/281 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [47]:
twitter_test_absa_polarity_preds = subpredictor.absa_model.polarity_model.predict(
    twitter_airlines_dataset["test"]["text"]
)
amazon_test_absa_polarity_preds = subpredictor.absa_model.polarity_model.predict(
    amazon_headphones_dataset["test"]["text"]
)
sst_test_absa_polarity_preds = subpredictor.absa_model.polarity_model.predict(sst_dataset["test"]["text"])

Batches:   0%|          | 0/92 [00:00<?, ?it/s]

Batches:   0%|          | 0/11 [00:00<?, ?it/s]

Batches:   0%|          | 0/70 [00:00<?, ?it/s]

### Twitter

In [50]:
print(simple_classification_report(twitter_airlines_dataset["test"]["label"], twitter_test_absa_polarity_preds))

              precision    recall  f1-score   support

    negative       0.91      0.50      0.65      1893
     neutral       0.29      0.55      0.38       615
    positive       0.51      0.87      0.64       420

    accuracy                           0.56      2928
   macro avg       0.57      0.64      0.56      2928
weighted avg       0.72      0.56      0.59      2928



In [51]:
print(simple_classification_report(twitter_airlines_dataset["test"]["label"], twitter_test_dacs_preds))

              precision    recall  f1-score   support

    negative       0.84      0.91      0.87      1893
     neutral       0.59      0.37      0.46       615
    positive       0.65      0.74      0.69       420

    accuracy                           0.77      2928
   macro avg       0.69      0.67      0.67      2928
weighted avg       0.76      0.77      0.76      2928



### Amazon

In [52]:
print(simple_classification_report(amazon_headphones_dataset["test"]["label"], amazon_test_absa_polarity_preds))

              precision    recall  f1-score   support

    negative       0.81      0.82      0.81       112
     neutral       0.11      0.04      0.05        28
    positive       0.86      0.94      0.90       181

    accuracy                           0.82       321
   macro avg       0.59      0.60      0.59       321
weighted avg       0.78      0.82      0.80       321



In [53]:
print(simple_classification_report(amazon_headphones_dataset["test"]["label"], amazon_test_dacs_preds))

              precision    recall  f1-score   support

    negative       0.77      0.88      0.82       112
     neutral       0.11      0.07      0.09        28
    positive       0.89      0.86      0.87       181

    accuracy                           0.80       321
   macro avg       0.59      0.60      0.59       321
weighted avg       0.78      0.80      0.79       321



### SST

In [54]:
print(simple_classification_report(sst_dataset["test"]["label"], sst_test_absa_polarity_preds))

              precision    recall  f1-score   support

    negative       0.79      0.69      0.74       912
     neutral       0.43      0.05      0.09       389
    positive       0.63      0.95      0.76       909

    accuracy                           0.68      2210
   macro avg       0.62      0.56      0.53      2210
weighted avg       0.66      0.68      0.63      2210



In [55]:
print(simple_classification_report(sst_dataset["test"]["label"], sst_test_dacs_preds))

              precision    recall  f1-score   support

    negative       0.74      0.77      0.75       912
     neutral       0.37      0.11      0.16       389
    positive       0.69      0.88      0.78       909

    accuracy                           0.70      2210
   macro avg       0.60      0.58      0.56      2210
weighted avg       0.66      0.70      0.66      2210



### Comparison

In [69]:
labels_dict = {
    "twitter": twitter_airlines_dataset["test"]["label"],
    "amazon": amazon_headphones_dataset["test"]["label"],
    "sst": sst_dataset["test"]["label"],
}
preds_dict = {
    "twitter": {"polarity": twitter_test_absa_polarity_preds, "dacs": twitter_test_dacs_preds},
    "amazon": {"polarity": amazon_test_absa_polarity_preds, "dacs": amazon_test_dacs_preds},
    "sst": {"polarity": sst_test_absa_polarity_preds, "dacs": sst_test_dacs_preds},
}

In [65]:
from sklearn.metrics import classification_report

x = classification_report([1, 0, 1], [0, 1, 1], output_dict=True)

In [75]:
compare_multiple(labels_dict, preds_dict, "accuracy")

,twitter,amazon,sst
polarity,0.564891,0.822430,0.684163
dacs,0.773224,0.797508,0.697285


In [74]:
compare_multiple(labels_dict, preds_dict, "macro f1")

,twitter,amazon,sst
polarity,0.556775,0.590196,0.528976
dacs,0.672026,0.594449,0.564449


In [76]:
compare_multiple(labels_dict, preds_dict, "weighted f1")

,twitter,amazon,sst
polarity,0.590039,0.797599,0.632138
dacs,0.758766,0.787661,0.658928


In [78]:
latex = compare_multiple(labels_dict, preds_dict, "weighted f1").to_latex()
from IPython.display import Latex, display

display(Latex(latex))

<IPython.core.display.Latex object>